---
format:
  html:
    code-line-numbers: true
    code-overflow: wrap
    code-block-bg: true
    code-block-border-left: true
    highlight-style: Arrow
---

# Benders Decomposition

In this chapter, we will explain the theories behind Benders decomposition and demonstrate its usage on a trial linear programming problem.
Keep in mind that Benders decomposition is not limited to solving linear programming problems.
In fact, it is one of the most powerful techniques to solve some large-scale mixed-integer linear programming problems.

In the following sections, we will go through the critical steps during the decomposition process when applying the algorithm on optimization problems represented in standard forms.
This is important as it helps build up the intuition of when we should consider applying Benders decomposition to a problem at hand.
Often times, recognizing the applicability of Benders decomposition is the most important and challenging step when solving an optimization problem.
Once we know that the problem structure is suitable to solve via Benders decomposition, it is straightforward to follow the decomposition steps and put it into work.

Generally speaking, Benders decomposition is a good solution candidate when the resulting problem is much easier to solve if some of the variables in the original problem are fixed.
We will illustrate this point using an example in the following sections.
In the optimization world, the first candidate that should come to mind when we say a problem is easy to solve is a linear programming formulation, which is indeed the case in Benders decomposition applications.

## The Decomposition Logic

To explain the reasoning of Benders decomposition, let us look at the standard form of linear programming problems that involve two vector variables, $\mathbf{x}$ and $\mathbf{y}$. Let $p$ and $q$ indicate the dimensions of $\mathbf{x}$ and $\mathbf{y}$, respectively.
Below is the original problem $\mathbf{P}$ we intend to solve.

\begin{align}
&(\mathbf{P}) &\quad \text{min.} &\quad \mathbf{c}^T \mathbf{x} + \mathbf{f}^T \mathbf{y} \\
& &\quad \text{s.t.} &\quad \mathbf{A} \mathbf{x} + \mathbf{B} \mathbf{y} = \mathbf{b} \\
& &\quad &\quad \mathbf{x} \geq 0, \mathbf{y} \geq 0
\end{align}

In this formulation, $\mathbf{c}$ and $\mathbf{f}$ in the objective function represent the cost coefficients associated with decision variables $\mathbf{x}$ and $\mathbf{y}$, respectively.
Both of them are column vectors of corresponding dimensions.
In the constraints, matrix $\mathbf{A}$ is of dimension $m \times p$, and matrix $\mathbf{B}$ is of dimension $m \times q$.
$\mathbf{b}$ is a column vector of dimension $m$.

Suppose the variable $\mathbf{y}$ is a complicating variable in the sense that the resulting problem is substantially easier to solve if the value of $\mathbf{y}$ is fixed.
In this case, we could rewrite problem $\mathbf{P}$ as the following form:

\begin{align}
\text{min.} &\quad \mathbf{f}^T \mathbf{y} + g(\mathbf{y}) \\
\text{s.t.} &\quad \mathbf{y} \geq 0
\end{align}

where $g(\mathbf{y})$ is a function of $\mathbf{y}$ and is defined as the subproblem $\mathbf{SP}$ of the form below:

\begin{align}
&(\mathbf{SP}) &\quad \text{min.} &\quad \mathbf{c}^T \mathbf{x} \\
& &\quad \text{s.t.} &\quad \mathbf{A} \mathbf{x}  = \mathbf{b} - \mathbf{B} \mathbf{y} \label{bd-cons1} \\
& &\quad &\quad \mathbf{x} \geq 0
\end{align}

Note that the $\mathbf{y}$ in constraint \eqref{bd-cons1} takes on some known values when the problem is solved and the only decision variable in the above formulation is $\mathbf{x}$.
The dual problem of $\mathbf{SP}$, $\mathbf{DSP}$, is given below.


\begin{align}
&(\mathbf{DSP}) &\quad \text{max.} &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u} \\
& &\quad \text{s.t.} &\quad \mathbf{A}^T \mathbf{u} \leq \mathbf{c} \label{bd-cons2} \\
& &\quad &\quad \mathbf{u}\  \text{unrestricted}
\end{align}

A key characteristic of the above $\mathbf{DSP}$ is that its solution space does not depend on the value of $\mathbf{y}$, which only affects the objective function.
According to the Minkowski’s representation theorem, any $\bar{\mathbf{u}}$ satisfying the constraints \eqref{bd-cons2} can be expressed as

\begin{align}
\bar{\mathbf{u}} = \sum_{j \in \mathbf{J}} \lambda_j \mathbf{u}_{j}^{point} + \sum_{k \in \mathbf{K}} \mu_k \mathbf{u}_k^{ray}
\end{align}

where $\mathbf{u}_j^{point}$ and $\mathbf{u}_k^{ray}$ represent an extreme point and extreme ray, respectively.
In addition, $\lambda_j \geq 0$ for all $j \in \mathbf{J}$ and $\sum_{j \in \mathbf{J}}\lambda_j = 1$, and $\mu_k \geq 0$ for all $k \in \mathbf{K}$.
It follows that the $\mathbf{DSP}$ is equivalent to 

\begin{align}
\text{max.} &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} (\sum_{j \in \mathbf{J}} \lambda_j \mathbf{u}_{j}^{point} + \sum_{k \in \mathbf{K}} \mu_k \mathbf{u}_k^{ray}) \\
\text{s.t.} &\quad \sum_{j \in \mathbf{J}}\lambda_j = 1 \\
&\quad \lambda_j \geq 0, \ \forall j \in \mathbf{J} \\
&\quad \mu_k \geq 0, \ \forall k \in \mathbf{K}
\end{align}

We can therefore conclude that

- The $\mathbf{DSP}$ becomes unbounded if any $\mathbf{u}_k^{ray}$ exists such that $(\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_k^{ray} > 0$.
Note that an unbounded $\mathbf{DSP}$ implies an infeasible $\mathbf{SP}$ and to prevent this from happening, we have to ensure that $(\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_k^{ray} \leq 0$ for all $k \in \mathbf{K}$.
- If an optimal solution to $\mathbf{DSP}$ exists, it must occur at one of the extreme points. Let $g$ denote the optimal objective value, it follows that $(\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_j^{point} \leq g$ for all $j \in \mathbf{J}$.

Based on this idea, the $\mathbf{DSP}$ can be reformulated as follows:

\begin{align}
\text{min.} &\quad g \\
\text{s.t.} &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_k^{ray} \leq 0, \ \forall j \in \mathbf{J} \label{bd-feas} \\
&\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_j^{point} \leq g, \ \forall k \in \mathbf{K} \label{bd-opt} \\
&\quad j \in \mathbf{J}, k \in \mathbf{K}
\end{align}

Constraints \eqref{bd-feas} are called **Benders feasibility cuts**, while constraints \eqref{bd-opt} are called **Benders optimality cuts**.
Now we are ready to define the Benders Master Problem ($\mathbf{BMP}$) as follows:

\begin{align}
&(\mathbf{BMP}) &\quad \text{min.} &\quad \mathbf{f}^T \mathbf{y} + g \\
& &\quad \text{s.t.} &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_k^{ray} \leq 0, \ \forall j \in \mathbf{J} \\
& &\quad &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_j^{point} \leq g, \ \forall k \in \mathbf{K} \\
& &\quad &\quad j \in \mathbf{J}, k \in \mathbf{K}, \mathbf{y} \geq 0
\end{align}

Typically $J$ and $K$ are too large to enumerate upfront and we have to work with subsets of them, denoted by $J_s$ and $K_s$, respectively. Hence we have the following Restricted Benders Master Problem ($\mathbf{RBMP}$):

\begin{align}
&(\mathbf{RBMP}) &\quad \text{min.} &\quad \mathbf{f}^T \mathbf{y} + g \\
& &\quad \text{s.t.} &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_k^{ray} \leq 0, \ \forall j \in \mathbf{J}_s \\
& &\quad &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_j^{point} \leq g, \ \forall k \in \mathbf{K}_s \\
& &\quad &\quad j \in \mathbf{J}, k \in \mathbf{K}, \mathbf{y} \geq 0
\end{align}

```{mermaid}
%%| label: fig-benders-flowchart
%%| fig-cap: Benders decomposition workflow

flowchart LR
   A[Start] --> B{Is}
    B -->|Yes| C[OK]
    C --> D[Rethink]
    D --> B
    B ---->|No| E[End]
```

## A linear programming example

In this section, we will first present a small linear programming problem and solve it directly using the Gurobi API in Python - *gurobipy*.
Then we will demonstrate the Benders decomposition approach on this artificial problem.
Lastly, we will provide an implementation to solve this problem in gurobipy.

### The original problem and its optimal solution

The linear program we examine here is devoid of any practical meaning and is solely used to demonstrate the solution process of Benders decomposition.
The problem is stated below, in which $\mathbf{x} = (x_1, x_2, x_3)$ and $\mathbf{y} = (y_1, y_2)$ are the decision variables.
We assume that $\mathbf{y}$ is the complicating variable.

\begin{align*}
    \text{min.} &\quad 8x_1 + 12x_2 +10x_3 + 15y_1 + 18y_2 \\
    \text{s.t.} &\quad 2x_1 + 3x_2 + 2x_3 + 4y_1 + 5y_2 = 300 \\
    &\quad 4x_1 + 2x_2 + 3x_3 + 2y_1 + 3y_2 = 228.75 \\
    &\quad 1x_1 + 2x_2 + 1x_3 + 1.5y_1 + 2y_2 = 150 \\
    &\quad 3x_1 + 2x_2 + 2x_3 + 1y_1 + 2y_2 = 180 \\
    &\quad x_i \geq 0, \ \forall i = 1, \cdots, 3 \\
    &\quad y_i \geq 0, \ \forall j = 1, 2
\end{align*}

In this example, $\mathbf{c}^T = (8, 12, 10)$, $\mathbf{f}^T = (15, 18)$ and $\mathbf{b}^T = (300, 228.75, 150, 180)$.
In addition,

\begin{equation*}
\mathbf{A} = 
\begin{bmatrix}
    2 & 3 & 2 \\
    4 & 2 & 3 \\
    1 & 2 & 1 \\
    3 & 2 & 2
\end{bmatrix}
\qquad
\mathbf{B} = 
\begin{bmatrix}
    4 & 5 \\
    2 & 3 \\
    1.5 & 2 \\
    1 & 2
\end{bmatrix}
\end{equation*}

We first use Gurobi to identify its optimal solution.

In [ ]:
#| echo: true
#| output: false

import gurobipy as gp
from gurobipy import GRB

# Create a new model
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()
model = gp.Model(env=env, name="original_problem")

# Decision variables
x1 = model.addVar(vtype=GRB.CONTINUOUS, name='x1')
x2 = model.addVar(vtype=GRB.CONTINUOUS, name='x2')
x3 = model.addVar(vtype=GRB.CONTINUOUS, name='x3')
y1 = model.addVar(vtype=GRB.CONTINUOUS, name='y1')
y2 = model.addVar(vtype=GRB.CONTINUOUS, name='y2')

# Objective function
model.setObjective(8*x1 + 12*x2 + 10*x3 + 15*y1 + 18*y2, 
                   GRB.MINIMIZE)

# Constraints
model.addConstr(2*x1 + 3*x2 + 2*x3 + 4*y1 + 5*y2 == 300)
model.addConstr(4*x1 + 2*x2 + 3*x3 + 2*y1 + 3*y2 == 220)
# model.addConstr(1*x1 + 2*x2 + 1*x3 + 1.5*y1 + 2*y2 == 150)
# model.addConstr(3*x1 + 2*x2 + 2*x3 + 1*y1 + 2*y2 == 180)

# Optimize the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f'x1 = {x1.X:.2f}')
    print(f'x2 = {x2.X:.2f}')
    print(f'x3 = {x3.X:.2f}')
    print(f'y1 = {y1.X:.2f}')
    print(f'y2 = {y2.X:.2f}')
    print(f"Total cost: {model.objVal:.2f}")
else:
    print("No solution found.")

# Close the Gurobi environment
model.dispose()
env.dispose()


The optimal solution and objective value are as follows.

```{{python}}
Optimal solution found!
x1 = 14.29
x2 = 0.00
x3 = 0.00
y1 = 0.00
y2 = 54.29
Total cost: 1091.43
```

### Benders decomposition

We first state the subproblem as follows:

\begin{align*}
    &(\mathbf{SP}) &\quad \text{min.} &\quad 8x_1 + 12x_2 +10x_3 \\
    &&\quad \text{s.t.} &\quad 2x_1 + 3x_2 + 2x_3 = 300 - 4y_1 - 5y_2 \\
    &&&\quad 4x_1 + 2x_2 + 3x_3 = 220 - 2y_1 + 3y_2 \\
    &&&\quad x_i \geq 0, \ \forall i = 1, \cdots, 3
\end{align*}

We define two dual variables $u_1$ and $u_2$ to associate with the two constraints in the subproblem.
The dual subproblem could then be stated as follows:

\begin{align*}
    &(\mathbf{DSP}) &\quad \text{max.} &\quad (300 - 4y_1 - 5y_2) u_1 + (220 - 2y_1 + 3y_2) u_2 \\
    &&\quad \text{s.t.} &\quad 2u_1 + 4u_2 \leq 8\\
    &&&\quad 3u_1 + 2u_2 \leq 12 \\
    &&&\quad 2u_1 + 3u_2 \leq 10 \\
    &&&\quad u_1, u_2\  \text{unrestricted}
\end{align*}

The *RBMP* can be stated as:

\begin{align*}
    &(\mathbf{RBMP}) &\quad \text{min.} &\quad 15 y_1 + 18 y_2 + g \\
    &&\quad \text{s.t.} &\quad  y_1, y_2 \geq 0 \\
    &&&\quad g \leq 0
\end{align*}

### Implementation

In [4]:
import gurobipy as gp
from gurobipy import GRB

env = gp.Env('benders')

Set parameter Username
Set parameter LogFile to value "benders"


In [8]:
# create restricted Benders master problem
rbmp = gp.Model(env=env, name='RBMP')

# create decision variables
y1 = rbmp.addVar(vtype=GRB.CONTINUOUS, lb=0, name='y1')
y2 = rbmp.addVar(vtype=GRB.CONTINUOUS, lb=0, name='y2')
g = rbmp.addVar(vtype=GRB.CONTINUOUS, lb=0, name='g')

# create objective
rbmp.setObjective(15*y1 + 18*y2 + g, GRB.MINIMIZE)

rbmp.optimize()

if rbmp.Status == GRB.OPTIMAL:
    print(f'y1 = {y1.X}')
    print(f'y2 = {y2.X}')
    print(f'g = {g.X}')
else:
    print(f'No solution found!')

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 3 columns and 0 nonzeros
Model fingerprint: 0xb00b2c2a
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  0.000000000e+00
y1 = 0.0
y2 = 0.0
g = 0.0


In [ ]:
# create subproblem
sp = gp.Model(env=env, name='SP')

In [7]:
# create dual subproblem
dsp = gp.Model(env=env, name='DSP')